# IPython Blockfolio

This is a simple "Blockfolio" using Python and the [CoinMarketCap API](https://coinmarketcap.com/api/).

It will show the net worth in USD of your Cryptocurrency investments as well as the relative share
of coins you own in comparison to the total coin supply of the corresponding digital asset.

First, the current exchanges rates are loaded, then combined with your investments, and finally displayed using Plotly.

This notebook can be found on [my github profile](https://github.com/SmokinCaterpillar/blockfolio).

Imports
--------

In [1]:
import pandas as pd

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

## Loading Data from CoinMarkteCap

In [2]:
limit = 200  # only load the top 200 currencies, if you have invested in smaller ones increase this limit
coin_market_cap_api = 'https://api.coinmarketcap.com/v1/ticker/?limit={}'.format(limit)
coin_market_cap_api

'https://api.coinmarketcap.com/v1/ticker/?limit=200'

In [3]:
# load the data using pandas and keep track of the time
now = pd.datetime.now()
market_data = pd.read_json(coin_market_cap_api)
market_data.head(10)

,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,symbol,total_supply
0,1.127230e+10,16769287,bitcoin,1514542160,244346957806,2.100000e+07,Bitcoin,-1.67,0.66,6.73,1.000000,14571.100000,1,BTC,16769287
1,2.397460e+09,96639172,ethereum,1514542149,71111258149,NaN,Ethereum,-1.92,2.58,9.42,0.051223,735.843000,2,ETH,96639172
2,3.596340e+09,38739144847,ripple,1514542140,65389739544,1.000000e+11,Ripple,4.49,26.34,59.31,0.000117,1.687950,3,XRP,99993093880
3,3.536340e+09,16881963,bitcoin-cash,1514542153,41489955138,2.100000e+07,Bitcoin Cash,-2.38,-6.27,-0.93,0.171080,2457.650000,4,BCH,16881963
4,3.006560e+09,54522208,litecoin,1514542141,13245080027,8.400000e+07,Litecoin,-1.82,-4.04,-3.39,0.016911,242.930000,5,LTC,54522208
5,1.078460e+08,25927070538,cardano,1514542155,11388543515,4.500000e+10,Cardano,0.89,14.62,13.02,0.000031,0.439253,6,ADA,31112483745
6,1.914670e+08,2779530283,iota,1514542151,9901381751,2.779530e+09,IOTA,-3.96,3.06,-4.04,0.000248,3.562250,7,MIOTA,2779530283
7,1.615940e+08,7781857,dash,1514542141,8510627831,1.890000e+07,Dash,-2.25,-0.06,-1.10,0.076130,1093.650000,8,DASH,7781857
8,4.768900e+07,8999999999,nem,1514542145,8313758999,NaN,NEM,0.34,6.00,13.82,0.000064,0.923751,9,XEM,8999999999
9,1.366960e+08,15529227,monero,1514542143,5696337741,NaN,Monero,-0.92,-1.36,13.37,0.025534,366.814000,10,XMR,15529227


## Your Blockfolio

Below you can fill in the types and amount of coins you have into the Python dictionary
with the format `'SYMBOL': value`.

In [4]:
# enter your coins below:

block_folio = {
    'BTC': 0.31337,
    'BCH': 0.01337,
    'ETH': 3,
    'LTC': 1.5,
    'MIOTA': 777,
    'XRP': 13,
    'DASH': 0.1, 
    'XMR': 1, 
    'LSK': 12, 
    'OMG': 1.1, 
    'XRB': 42, 
    'PIVX': 123, 
    'ARK': 22, 
    'NEO': 7, 
    'GAS': 0.01,
    'STEEM': 499,
}

# display your blockfolio
block_folio = pd.DataFrame(list(block_folio.items()), columns=['symbol', 'amount'])
block_folio.head(len(block_folio))

,symbol,amount
0,LSK,12.00000
1,GAS,0.01000
2,XRP,13.00000
3,NEO,7.00000
4,XMR,1.00000
5,PIVX,123.00000
6,MIOTA,777.00000
7,BTC,0.31337
8,ARK,22.00000
9,ETH,3.00000


### Combining the Blockfolio and the API Data

In [5]:
# merge the API and blockfolio data and sort by investment value
merged_data = block_folio.merge(market_data, how='left')
merged_data['value_usd'] = merged_data.amount * merged_data.price_usd
merged_data['coinshare'] = merged_data.amount / merged_data.available_supply
merged_data = merged_data.sort_values('value_usd', ascending=False)
merged_data.head()

,symbol,amount,24h_volume_usd,available_supply,id,last_updated,market_cap_usd,max_supply,name,percent_change_1h,percent_change_24h,percent_change_7d,price_btc,price_usd,rank,total_supply,value_usd,coinshare
7,BTC,0.31337,1.127230e+10,16769287,bitcoin,1514542160,244346957806,2.100000e+07,Bitcoin,-1.67,0.66,6.73,1.000000,14571.10000,1,16769287,4566.145607,1.868714e-08
6,MIOTA,777.00000,1.914670e+08,2779530283,iota,1514542151,9901381751,2.779530e+09,IOTA,-3.96,3.06,-4.04,0.000248,3.56225,7,2779530283,2767.868250,2.795436e-07
9,ETH,3.00000,2.397460e+09,96639172,ethereum,1514542149,71111258149,NaN,Ethereum,-1.92,2.58,9.42,0.051223,735.84300,2,96639172,2207.529000,3.104331e-08
11,STEEM,499.00000,4.653600e+06,246344041,steem,1514542146,703883756,NaN,Steem,-0.82,0.22,6.18,0.000199,2.85732,41,263318135,1425.802680,2.025622e-06
5,PIVX,123.00000,7.378790e+06,55233563,pivx,1514542146,620587747,NaN,PIVX,1.79,-1.53,43.96,0.000782,11.23570,44,55233563,1381.991100,2.226907e-06


### Your Blockfolio in numbers and graphs!


In [6]:
networth = 'Your blockfolio is currently (i.e {}) worth {:.2f} USD!'.format(now.strftime('%Y-%m-%d %I:%M%p'), 
                                                                        merged_data.value_usd.sum())
print(networth)

Your blockfolio is currently (i.e 2017-12-29 11:09AM) worth 14759.14 USD!


#### Net Worth

Let's plot the worth of each asset in descending order:

In [7]:
marker = dict(color='rgb(158,202,225)',
              line=dict(color='rgb(8,48,107)', width=1.5))

layout = go.Layout(title='Blockfolio Networth in USD',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='USD'))
              
value_chart=go.Bar(x=merged_data.symbol, y=merged_data.value_usd, marker=marker)

fig = go.Figure(data=[value_chart], layout=layout)
py.iplot(fig)

#### Fraction of total Supply
Finally, that looks at the relative sizes of your investment. 
This gives you an idea or visualizes in which cryptocurrency you believe in the most. 
The graph below shows you fractional share of the the current crypto's total supply.

In [8]:
marker = dict(color='rgb(258,202,125)', line=dict(color='rgb(8,48,107)', width=1.5,))

layout = go.Layout(title='Relative Blockfolio Size',
                   xaxis=dict(title='Cryptocurrency'),
                   yaxis=dict(title='Fraction of total Supply'))

share_chart=go.Bar(x=merged_data.symbol, y=merged_data.coinshare, marker=marker)

fig = go.Figure(data=[share_chart], layout=layout)
py.iplot(fig)